In [2]:
import tensorflow as tf
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import cv2

model_dir = './sr_models/xlsr_quantized.tflite'
interpreter = tf.lite.Interpreter(model_path=model_dir)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"

init_image = download_image(img_url).resize((128, 128))

2024-05-30 11:03:08.319956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 11:03:08.320006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 11:03:08.321107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 11:03:08.327883: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 11:03:09.176221: W tensorflow/compiler/tf2

## Check input_details and fis the scale value and zero point value

In [3]:
input_details

[{'name': 'image',
  'index': 0,
  'shape': array([  1, 128, 128,   3], dtype=int32),
  'shape_signature': array([  1, 128, 128,   3], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.003921568859368563, -128),
  'quantization_parameters': {'scales': array([0.00392157], dtype=float32),
   'zero_points': array([-128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [4]:
scale = 0.003921568859368563
zero_point = -128

# Convert image to numpy array and expand dimensions to match model input
init_image_array = np.array(init_image).astype(np.float32)  # 이미지 값을 float32로 변환
init_image_array = np.expand_dims(init_image_array, axis=0)  # 배치 차원 추가

# Quantize the image
init_image_quantized = np.round(init_image_array / scale + zero_point).astype(np.int8)

## Model Inference

In [5]:
# Set input tensor
interpreter.set_tensor(input_details[0]['index'], init_image_quantized)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

## Check output_details and fis the scale value and zero point value

In [7]:
output_details

[{'name': 'output_0',
  'index': 43,
  'shape': array([  1, 512, 512,   3], dtype=int32),
  'shape_signature': array([  1, 512, 512,   3], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.003913909196853638, -128),
  'quantization_parameters': {'scales': array([0.00391391], dtype=float32),
   'zero_points': array([-128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [6]:
output_scale = 0.003913909196853638
output_zero_point = -128

def dequantize_output(output_tensor, scale, zero_point):
    return (output_tensor.astype(np.float32) - zero_point) * scale

output_dequantized = dequantize_output(output_data, output_scale, output_zero_point)
output_uint8 = np.clip(output_dequantized * 255, 0, 255).astype(np.uint8)
output_image = output_uint8[0]

#### OPTIONAL ####
output_image = cv2.bitwise_not(output_image)



output_pil_image = Image.fromarray(output_image)
output_pil_image.save("output_image.png")